## Import libraries

In [1]:
import json
import numpy as np
import random
import torch
import time
import pandas as pd
import gc

Making the notebook deterministic

In [2]:
def fix_random(seed: int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


fix_random(seed=42)

In [3]:
data_path = "/kaggle/input/squad0"

This function loads a little SQuAD dataset for evaluation.

In [4]:
def load(path):
    data = []
    with open(f"{data_path}/squad_val_1k.jsonl", "r") as file:
          for line in file:
              try:
                  data.append(json.loads(line))
              except Exception as e:
                  print("json processing exception", e)
                  continue
    return data

In [5]:
data = load('squad_val_1k.jsonl')
print(data[0])

{'context': 'The Panthers beat the Seattle Seahawks in the divisional round, running up a 31–0 halftime lead and then holding off a furious second half comeback attempt to win 31–24, avenging their elimination from a year earlier. The Panthers then blew out the Arizona Cardinals in the NFC Championship Game, 49–15, racking up 487 yards and forcing seven turnovers.', 'prompt': 'How many yards did the Panthers get for the division championshipt game?', 'response': '487'}


This functions necessary for evaluating the models. More description below.

In [6]:
#This function generate the prompt. If we set the few_shot parameter to True, then it will generate
#a prompt, which uses few-shot prompting technique, otherwise not. Provide a possibility for two-shot
#prompting, because the three-shot prompting generate so long contexts, that these are longer
#than the max_context_window of the model.
def generate_prompt(few_shot, question_with_context):
    questions = []
    if few_shot == 'two-shot':
        two_shot_prompting = [
        {
            "context" : "On April 12, 1961, Soviet cosmonaut Yuri Gagarin became the first person to fly in space, reinforcing American fears about being left behind in a technological competition with the Soviet Union. At a meeting of the US House Committee on Science and Astronautics one day after Gagarin's flight, many congressmen pledged their support for a crash program aimed at ensuring that America would catch up. Kennedy was circumspect in his response to the news, refusing to make a commitment on America's response to the Soviets.",
            "question": "How many days after Gagarin's flight did the US House Committee on Science and Astronautics meet?",
            "answer": "one day",
        },
        {
            "context" : "The customary law of Normandy was developed between the 10th and 13th centuries and survives today through the legal systems of Jersey and Guernsey in the Channel Islands. Norman customary law was transcribed in two customaries in Latin by two judges for use by them and their colleagues: These are the Très ancien coutumier (Very ancient customary), authored between 1200 and 1245; and the Grand coutumier de Normandie (Great customary of Normandy, originally Summa de legibus Normanniae in curia laïcali), authored between 1235 and 1245.",
            "question": "Where are Jersey and Guernsey?",
            "answer": "Channel Islands",
        }
        ]
        for item in question_with_context:
            prompt = f"You are a question answering bot. Your task is to answer the questions based on the appropriate contexts and your own knowledge. Your answers should contain only the most important things and should be as short as possible."
            prompt = f"{prompt}\n\n" + "\n\n".join([f"{p['context']}\n\nQ: {p['question']}\n\nA: {p['answer']}" for p in two_shot_prompting])
            prompt = f"{prompt}\n\n{item}\n\nA:"
            questions.append(prompt)
    elif few_shot == 'three-shot':
        three_shot_prompting = [
        {
            "context" : "In 1993, the FCC repealed the Financial Interest and Syndication Rules, once again allowing networks to hold interests in television production studios. That same year, Capital Cities/ABC purchased the French animation studio DIC Entertainment; it also signed an agreement with Time Warner Cable to carry its owned-and-operated television stations on the provider's systems in ABC O&O markets. By that year, ABC had a total viewership share of 23.63% of American households, just below the limit of 25% imposed by the FCC.",
            "question": "What French animation studio did ABC purchase in 1993?",
            "answer": "DIC Entertainment",
        },
        {
            "context" : "On April 12, 1961, Soviet cosmonaut Yuri Gagarin became the first person to fly in space, reinforcing American fears about being left behind in a technological competition with the Soviet Union. At a meeting of the US House Committee on Science and Astronautics one day after Gagarin's flight, many congressmen pledged their support for a crash program aimed at ensuring that America would catch up. Kennedy was circumspect in his response to the news, refusing to make a commitment on America's response to the Soviets.",
            "question": "How many days after Gagarin's flight did the US House Committee on Science and Astronautics meet?",
            "answer": "one day",
        },
        {
            "context" : "The customary law of Normandy was developed between the 10th and 13th centuries and survives today through the legal systems of Jersey and Guernsey in the Channel Islands. Norman customary law was transcribed in two customaries in Latin by two judges for use by them and their colleagues: These are the Très ancien coutumier (Very ancient customary), authored between 1200 and 1245; and the Grand coutumier de Normandie (Great customary of Normandy, originally Summa de legibus Normanniae in curia laïcali), authored between 1235 and 1245.",
            "question": "Where are Jersey and Guernsey?",
            "answer": "Channel Islands",
        }
        ]
        for item in question_with_context:
            prompt = f"You are a question answering bot. Your task is to answer the questions based on the appropriate contexts and your own knowledge. Your answers should contain only the most important things and should be as short as possible."
            prompt = f"{prompt}\n\n" + "\n\n".join([f"{p['context']}\n\nQ: {p['question']}\n\nA: {p['answer']}" for p in three_shot_prompting])
            prompt = f"{prompt}\n\n{item}\n\nA:"
            questions.append(prompt)
    else:
        for item in question_with_context:
            prompt = f"You are a question answering bot. Your task is to answer the questions based on the appropriate contexts and your own knowledge. Your answers should contain only the most important things and should be as short as possible."
            prompt = f"{prompt}\n\n{item}\n\nA:"
            questions.append(prompt)
    return questions

#This function generate the text for the prompt.
def run_with_SQuAD(model, tokenizer, question, few_shot):
    prompt = generate_prompt(few_shot, question)
    
    inputs = tokenizer(prompt, padding=True, truncation=True, return_tensors="pt").to(0)
    model.to(0)
    
    out = model.generate(
        input_ids=inputs['input_ids'],
        max_new_tokens= 128,
        attention_mask=inputs['attention_mask'],
        eos_token_id=tokenizer.eos_token_id
    )
    
    answers = []
    num_tokens = []
    
    decoded = tokenizer.batch_decode(out)

    for decodedText, promptText in zip(decoded, prompt):
        cleaned = decodedText.replace("<|endoftext|>", "")
        num_tokens.append(len(cleaned) - len(promptText))
        cleaned = cleaned.replace(promptText, "")
        answer = cleaned.split("\n\n")[0].strip()
        answers.append(answer)

    return answers, num_tokens

#In this function we making the evaluation for every single datapoint. During the generation we
#are making a statistics about the generation.
def eval(data, model, tokenizer, output_file, few_shot, batch_size):
    numberOfTokens = 0
    timeOfStart = 0
    results = []
    total_qs = len(data)
    correct = 0
    i = 0

    for batch_start in range(0, total_qs, batch_size):
        batch_end = min(batch_start + batch_size, total_qs)
        batch_data = data[batch_start:batch_end] 

        # Prepare batch inputs
        questions = [item['prompt'] for item in batch_data]
        contexts = [item['context'] for item in batch_data]
        answers = [item['response'] for item in batch_data]
        inputs = [f"Question: {que}\n\nContext: {ctx}\n\nQuestion: {que}" for ctx, que in zip(contexts, questions)]
        
        # Generate outputs in a batch
        start_time = time.time()
        batch_guesses, batch_num_tokens = run_with_SQuAD(model, tokenizer, inputs, few_shot)
        end_time = time.time()
        timeOfStart += end_time - start_time

        # Process results 
        for guess, num_tokens, answer, question, context in zip(batch_guesses, batch_num_tokens, answers, questions, contexts):
            if guess and guess[-1] == '.' and answer[-1] != '.':
                guess = guess[:-1]
            numberOfTokens += num_tokens
            tkps = num_tokens / (end_time - start_time)
            is_correct = (answer.strip().lower() == guess.strip().lower())
            print(f"Question {i+1}/{total_qs}")
            print(f"Q: {question}")
            print(f"A: {answer}")
            print(f"?: {guess}")
            if is_correct:
                print(f"✅")
                correct += 1
            else:
                print(f"❌")
            print("="*80)
            result = {
                "idx": i,
                "question": question,
                "context": context,
                "answer": answer,
                "guess": guess,
                "is_correct": is_correct,
                "time": end_time - start_time,
                "num_tokens": num_tokens,
                "tokens_per_sec": tkps
            }
            results.append(result)
            i += 1
            
            if len(results) % 20 == 0:
                write_results(results, output_file)

    print(f"Accuracy: {correct / total_qs * 100}% -- {correct} correct and {total_qs - correct} incorrect")
    print(f"Number of tokens generated: {numberOfTokens} -- Time: {timeOfStart} -- Tokens-Per-Sec: {numberOfTokens / timeOfStart}")
    
#This function writes out the results.
def write_results(results, output_file):
    df = pd.DataFrame(results)
    df = df[["idx", "question", "context", "answer", "guess", "is_correct", "time", "num_tokens", "tokens_per_sec"]]
    print(f"Writing {output_file}")
    df.to_json(output_file, orient="records", lines=True)

## Transformer-based LLM - GPT2 124m model

In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer_gpt2 = GPT2Tokenizer.from_pretrained('gpt2', padding_side='left')
model_gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer_gpt2.eos_token = "<|endoftext|>"
tokenizer_gpt2.pad_token = tokenizer_gpt2.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
eval(data[:100], model_gpt2, tokenizer_gpt2, "evalGPT2_124M.json", few_shot="none", batch_size=32)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question 1/100
Q: How many yards did the Panthers get for the division championshipt game?
A: 487
?: The Panthers beat the Seattle Seahawks in the divisional round, running up a 31–0 halftime lead and then holding off a furious second half comeback attempt to win 31–24, avenging their elimination from a year earlier. The Panthers then blew out the Arizona Cardinals in the NFC Championship Game, 49–15, racking up 487 yards and forcing seven turnovers
❌
Question 2/100
Q: What caused Jacksonville's tourism to become less desirable at the latter half of the 19th century?
A: yellow fever outbreaks
?: The first major change in the city's tourism was the construction of the new downtown. The new downtown was built on the site of the former Confederate Soldiers and Sailors Home. The new downtown was built on the site of the former Confederate Soldiers and Sailors Home. The new downtown was built on the site of the former Confederate Soldiers and Sailors Home. The new downtown was built on the 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question 33/100
Q: What type of materials inside the cabin were removed to help prevent more fire hazards in the future?
A: flammable cabin and space suit materials
?: The Block II crew members would wear modified, fire-resistant Block II space suits, and would be designated by the Block II titles, regardless of whether a LM was present on the flight or not
❌
Question 34/100
Q: How many nations contain "Amazonas" in their names?
A: States or departments in four nations contain "Amazonas" in their names.
?: The Amazon is a tropical rainforest, with a total area of over 1,000 square kilometres (1,000 sq mi). The Amazon is a tropical rainforest with a total area of over 1,000 square kilometres (1,000 sq mi). The Amazon is a tropical rainforest with a total area of over 1,000 square kilometres (1,000 sq mi).
❌
Question 35/100
Q: What was Tesla's position with Westinghouse?
A: consultant
?: Tesla was a pioneer in the electric motor industry. He was the first to develop a motor that could be

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question 65/100
Q: What ABC action series went up against NBC's variety shows in Fall 1957?
A: Zorro
?: The first series of the series, "The Big Bang Theory", was broadcast on ABC in the fall of 1957. The second series, "The Big Bang Theory: The Next Generation", was broadcast on ABC in the fall of 1958. The third series, "The Big Bang Theory: The Next Generation: The Next Generation", was broadcast on ABC in the fall of 1959
❌
Question 66/100
Q: What was the significance of victory at Forth Niagara for British?
A: cut off the French frontier forts further to the west and south
?: The victory at Forth Niagara was a major victory for the British. The British were able to hold the French and French-speaking territories of the North Atlantic for a long time, and they were able to hold the British-speaking territories of the South Atlantic for a long time. The British were able to hold the British-speaking territories of the South Atlantic for a long time, and they were able to hold the Br

In [9]:
eval(data[:100], model_gpt2, tokenizer_gpt2, "evalGPT2_124M_few_shot.json", few_shot="two-shot", batch_size=32)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question 1/100
Q: How many yards did the Panthers get for the division championshipt game?
A: 487
?: The Panthers beat the Seattle Seahawks in the divisional round, running up a 31–0 halftime lead and then holding off a furious second half comeback attempt to win 31–24, avenging their elimination from a year earlier. The Panthers then blew out the Arizona Cardinals in the NFC Championship Game, 49–15, racking up 487 yards and forcing seven turnovers. Question: How many yards did the Panthers get for the division championshipt game?
❌
Question 2/100
Q: What caused Jacksonville's tourism to become less desirable at the latter half of the 19th century?
A: yellow fever outbreaks
?: The Great Depression
❌
Question 3/100
Q: What are pharmacists in the United Kingdom being increasingly paid for?
A: medicine use reviews
?: The United Kingdom has a long history of pharmacists in the United Kingdom. In the early 19th century, the British Medical Association (BMA) was established to provide pharm

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question 33/100
Q: What type of materials inside the cabin were removed to help prevent more fire hazards in the future?
A: flammable cabin and space suit materials
?: The Block II design already called for replacement of the Block I plug-type hatch cover with a quick-release, outward opening door. NASA discontinued the manned Block I program, using the Block I spacecraft only for unmanned Saturn V flights. Crew members would also exclusively wear modified, fire-resistant Block II space suits, and would be designated by the Block II titles, regardless of whether a LM was present on the flight or not
❌
Question 34/100
Q: How many nations contain "Amazonas" in their names?
A: States or departments in four nations contain "Amazonas" in their names.
?: The Amazon is a tropical rainforest, with a total area of over 1,000 square kilometres (1,000 sq mi). The Amazon is a tropical rainforest with a total area of over 1,000 square kilometres (1,000 sq mi). The Amazon is a tropical rainforest wi

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question 65/100
Q: What ABC action series went up against NBC's variety shows in Fall 1957?
A: Zorro
?: The ABC action series, which was broadcast on the ABC-owned Channel 4, was a series of short stories that were based on the popular television series "The Good Wife". The series was based on the popular television series "The Good Wife", which was broadcast on the ABC-owned Channel 4, and was based on the popular television series "The Good Wife", which was broadcast on the ABC-owned Channel 4. The series was based on the popular television series "The Good Wife", which was broadcast on the ABC-owned Channel 4, and was based on the popular television series "The Good Wife", which was broadcast on the ABC
❌
Question 66/100
Q: What was the significance of victory at Forth Niagara for British?
A: cut off the French frontier forts further to the west and south
?: The British victory at Forth Niagara was the first major victory for the British in the Annus Mirabilis of 1759. The British w

In [10]:
model_gpt2 = None
tokenizer_gpt2 = None

gc.collect()

30

## Transformer based LLM - GPT-Neo 2.7B 

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer_gpt = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B", padding_side='left')
model_gpt = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B")
tokenizer_gpt.eos_token = "<|endoftext|>"
tokenizer_gpt.pad_token = tokenizer_gpt.eos_token

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

In [12]:
eval(data[:100], model_gpt, tokenizer_gpt, "evalGPT_Neo.json", few_shot="none", batch_size=4)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question 1/100
Q: How many yards did the Panthers get for the division championshipt game?
A: 487
?: The Panthers got 487 yards for the divisional round game
❌
Question 2/100
Q: What caused Jacksonville's tourism to become less desirable at the latter half of the 19th century?
A: yellow fever outbreaks
?: The city's tourism was dealt major blows in the late 19th century by yellow fever outbreaks. In addition, extension of the Florida East Coast Railway further south drew visitors to other areas
❌
Question 3/100
Q: What are pharmacists in the United Kingdom being increasingly paid for?
A: medicine use reviews
?: The remuneration of pharmacists in the United Kingdom is increasing. In the United Kingdom, pharmacists are being paid for their services in the following ways:
❌
Question 4/100
Q: Geoglyphs dating to what period were found in deforested land along the Amazon River?
A: AD 0–1250
?: The first European to travel the length of the Amazon River was Francisco de Orellana in 1542. The

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question 5/100
Q: Who does a gender pay gap tend to favor?
A: males in the labor market
?: Women
❌
Question 6/100
Q: What is the annual construction industry revenue in 2014?
A: $960 billion
?: $960 billion
✅
Question 7/100
Q: Which sculpture by Michelangelo has a full-size replica in the Cast Courts?
A: David
?: The David
❌
Question 8/100
Q: Richard Allen and Absalom Jones were licensed by St. George's Church in what year?
A: 1784
?: 1784
✅


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Question 85/100
Q: Issues dealt with at Westminster are not ones who is able to deal with?
A: Scottish Parliament
?: The Scottish Parliament is unable to legislate on such issues that are reserved to, and dealt with at, Westminster (and where Ministerial functions usually lie with UK Government ministers). These include abortion, broadcasting policy, civil service, common markets for UK goods and services, constitution, electricity, coal, oil, gas, nuclear energy, defence and national security, drug policy, employment, foreign policy and relations with Europe, most aspects of transport safety and regulation, National Lottery, protection of borders, social security and stability of UK's fiscal, economic and monetary system
❌
Question 86/100
Q: What was the wedding date?
A: 12 May 1191
?: 12 May 1191
✅
Question 87/100
Q: What did the development of this fertile soil provide in hostile environment?
A: agriculture and silviculture
?: The development of this fertile soil allowed agriculture

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question 89/100
Q: Which country does the Rhine encounter it's main tributaries?
A: Germany
?: Germany
✅
Question 90/100
Q: What French animation studio did ABC purchase in 1993?
A: DIC Entertainment
?: DIC Entertainment
✅
Question 91/100
Q: What rive joins the Rhine in Duisburg? 
A: Ruhr
?: The Rhine in Duisburg is joined by the Ruhr, which is a tributary of the Rhine. The Ruhr is a tributary of the Rhine, which is a tributary of the Rhine. The Ruhr is a tributary of the Rhine, which is a tributary of the Rhine
❌
Question 92/100
Q: Which animal's venom did Pierre-Louis Moreau de Maupertuis work with?
A: scorpion
?: The venom of the scorpion, which is a type of arthropod
❌


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question 93/100
Q: In the capabilities approach, grow and income are considered a means to an end rather than what?
A: the end itself
?: In the capabilities approach, income is considered a means to an end rather than what?
❌
Question 94/100
Q: Where are Jersey and Guernsey
A: Channel Islands
?: Jersey and Guernsey are two of the Channel Islands. They are both located in the English Channel, between England and France
❌
Question 95/100
Q: What was the boat called?
A: teleautomaton
?: The boat was called the "Tesla-Boat."
❌
Question 96/100
Q: What did the SNP publicly opine about the oil revenues?
A: not benefitting Scotland as much as they should
?: The SNP argued that the revenues from the oil were not benefitting Scotland as much as they should. The combined effect of these events led to Prime Minister Wilson committing his government to some form of devolved legislature in 1974. However, it was not until 1978 that final legislative proposals for a Scottish Assembly were passed by th

In [8]:
eval(data[:100], model_gpt, tokenizer_gpt, "evalGPT_Neo.json", few_shot="three-shot", batch_size=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question 1/100
Q: How many yards did the Panthers get for the division championshipt game?
A: 487
?: 487 yards
❌
Question 2/100
Q: What caused Jacksonville's tourism to become less desirable at the latter half of the 19th century?
A: yellow fever outbreaks
?: Yellow fever outbreaks
✅


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question 3/100
Q: What are pharmacists in the United Kingdom being increasingly paid for?
A: medicine use reviews
?: pharmacists in the United Kingdom are being increasingly paid for their services
❌
Question 4/100
Q: Geoglyphs dating to what period were found in deforested land along the Amazon River?
A: AD 0–1250
?: AD 0–1250
✅


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Question 93/100
Q: In the capabilities approach, grow and income are considered a means to an end rather than what?
A: the end itself
?: The capabilities approach – sometimes called the human development approach – looks at income inequality and poverty as form of “capability deprivation”. Unlike neoliberalism, which “defines well-being as utility maximization”, economic growth and income are considered a means to an end rather than the end itself. Its goal is to “wid[en] people’s choices and the level of their achieved well-being” through increasing functionings (the things a person values doing), capabilities (the freedom to enjoy functionings) and agency (the ability to pursue valued goals)
❌
Question 94/100
Q: Where are Jersey and Guernsey
A: Channel Islands
?: Channel Islands
✅


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question 95/100
Q: What was the boat called?
A: teleautomaton
?: Tesla's "Teleautomaton"
❌
Question 96/100
Q: What did the SNP publicly opine about the oil revenues?
A: not benefitting Scotland as much as they should
?: The SNP argued that the revenues from the oil were not benefitting Scotland as much as they should
❌


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question 97/100
Q: How many days after Gagarin's flight did the US House Committee on Science and Astronautics meet?
A: one day
?: one day
✅
Question 98/100
Q: How can function problems typically be restated?
A: decision problems
?: The set of triples (a, b, c) such that the relation a × b = c holds. Deciding whether a given triple is a member of this set corresponds to solving the problem of multiplying two numbers
❌
Question 99/100
Q: What does Fortiter Defendit Triumphans mean?
A: Triumphing by a brave defence
?: Fortiter Defendit Triumphans
❌
Question 100/100
Q: Recent studies believe  that ctenophores are the sister lineage to what?
A: all other animals
?: Cnidaria
❌
Writing evalGPT_Neo.json
Accuracy: 33.0% -- 33 correct and 67 incorrect
Number of tokens generated: 51982 -- Time: 608.1826891899109 -- Tokens-Per-Sec: 85.47102856419532


In [9]:
tokenizer_gpt = None
model_gpt = None

gc.collect()

390